# Convolutional and recurrent neural networks

In the last lecture we introduced deep learning, talked about the need to go deep, and trained a deep neural network using `sklearn` and then using `tensorflow`. The networks that we trained then were examples of **dense** or **fully connected** networks, meaning that every feature in the previous layer is connected to every feature in the current layer. Because of this property, dense neworks are very likely to overfit and have a lot of parameters. Too many parameters means that we run into computational limits as we try to go deeper. 

In this lecture, we want to expand on the idea of deep learning by proposing some modifications to dense networks. The first modification will introduce the idea of **convolutional neuron networks** which learn features in such a way that we take **locality** into account. The next modification will introduce **recurrent neural networks** which learn features when we have sequential data (i.e. data where order matters). Both ideas rely on **shared parameters** to greatly reduce the number of parameters in a neural networks, allowing us to **go deeper**.

## MNIST and convolutional networks

Let's begin by loading the MNIST data again.

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# from tensorflow.keras.utils import to_categorical
# y_train_onehot = to_categorical(y_train, num_classes = 10)
# y_test_onehot = keras.utils.to_categorical(y_test, num_classes = 10)

The following helper function will later be used to plot the training and validation data accuracy and loss for the models we train.

In [ ]:
import os
import matplotlib.pyplot as plt

def plot_train_test_performance(model):

    fig = plt.figure(figsize = [10, 10])
    plt.subplot(2,1,1)
    plt.plot(model.history.history['accuracy'])
    plt.plot(model.history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc = 'lower right')

    plt.subplot(2,1,2)
    plt.plot(model.history.history['loss'])
    plt.plot(model.history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc = 'upper right')
    plt.tight_layout()
    


The 2D convolutional neural networks we work with here expect the input data to be images. However, images are usually not 2D objects but 3D objects: the first two dimensions encode the size of the image, and the third dimension is for the **channel**. For example, a colored image oncoded as **RGB** (red, green, blue) has three channels. A greyscale image like the ones in the MNIST dataset have only one channel, but we still need the data to be 3D. Of course, the dataset itself then needs another dimension for the training example. This means our data has the following dimensionality:

$$\text{training examples} \times \text{row pixels} \times \text{column pixels} \times \text{channel}$$

We need to make sure that our data abides by the dimensionality above, and we use `reshape` for that. We also normalize the data as we did in the previous lecture by dividing the pixel values by 255 so the pixel values are between 0 and 1.

In [ ]:
print('x_train shape (before):', X_train.shape)
print('x_test shape (before):', X_test.shape)

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

print('x_train shape (after):', X_train.shape)
print('x_test shape (after):', X_test.shape)

X_train = x_train.astype('float32')
X_test = x_test.astype('float32')
X_train /= 255
X_test /= 255

As usual, before training a new model, let's first train a baseline model so we can compare performances. As our baseline, we choose a neural network with only a single hidden layer with 128 neurons in it.

In [ ]:
model_baseline = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dense(64,activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

We now compile our model. To speed up the training, we use the Adam optimizer, which is more efficient than vanilla SGD.

In [ ]:
model_baseline.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(0.01),
    metrics = ['accuracy'],
)

We now train our model, specifying a mini-batch size of 128 and a total of 20 epochs.

In [ ]:
model_baseline.fit(
    x_train, 
    y_train, 
    batch_size = 128,
    epochs = 20,
    verbose = 1,
    validation_data = (x_test, y_test)
)

And we now plot accuracy and loss for the training and validation data, epoch by epoch.

In [ ]:
plot_train_test_performance(model_baseline)

Time to train a convolutional neural network. Foretunately, there's very little we need to change in the code: The only changes show up in the architecture of the model, where we now use `Conv2D` and `MaxPool2D` layers initially. Note that the flat layers don't disappear. Instead they get pushed to the end of the network.

In [ ]:
model_cnn = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu', input_shape = (28, 28, 1)),
  tf.keras.layers.MaxPool2D((2, 2)),
  # tf.keras.layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
  # tf.keras.layers.MaxPool2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation = 'relu'),
  tf.keras.layers.Dense(10, activation = 'softmax')
])

model_cnn.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(0.01),
    metrics = ['accuracy'],
)

model_cnn.fit(
    x_train, 
    y_train, 
    batch_size = 128,
    epochs = 20,
    verbose = 1,
    validation_data = (x_test, y_test)
)

In [ ]:
plot_train_test_performance(model_cnn)

### Exercise

- Train the model for another two epochs by copying and pasting the **training portion** of the code above in the cell below. What is the starting accuracy at epoch 1? What does this tell you about the model?

When you create a model with `keras`, assuming your model is called `model`, you can use `model.summary()` to print some useful information about the model. You can print the model summary prior to training, in order to have an idea of the complexity of the model.

- Print the model summary for both the dense neural network and the CNN we trained above. How many parameters do each have?

### End of exercise

If we wanted to save the trained model we can use the `save_weights` method. This is useful in case we (1) need to later train the model further, (2) fine-tune the model to a more specific task (called transfer learning), or (3) load the model in a scoring environment to get predictions.

In [ ]:
model.save_weights("model_digit.h5")
print("Saved model to disk")

# IMBD and recurrent networks

We now shift focus to recurrent neural networks. Recurrent networks become sort of demystified once we realize that they are just a regular neural network unrolled. Using `keras` we don't have to manually do the unrolling: the details are handled under the hood.

For the next example, we use the IMDB dataset which consists of 25000 **movie reviews** for training and 25000 for testing. The label is the user's **sentiment** about the movie: positive or negative. We use recurrent neural networks to train a binary classifier to do sentiment analysis.

In [ ]:
imbd = tf.keras.datasets.imdb

After loading the data, we will split it into trining and testing. Note that we can decide the size of the corpus to use. This can be handy when we need to try our code and want to keep computations under control. By specifying for example to use a corpus with 10,000 words, we limit the reviews to use the most common 10,000 words. As we will later see, any word in the review that doesn't make the cut is replaced by a generic `<UNK>` marker.

In [ ]:
num_of_words = 10000
(x_train, y_train), (x_test, y_test) = imbd.load_data(num_words = num_of_words)

We now check the first row of the data to see what it looks like.

In [ ]:
print(x_train[0])
print(y_train[0])

That doesn't look much like a movie review, does it? The reason is that the data that keras is presenting us with has already been **pre-processed**, meaning that instead of looking at the raw movie review, each review has been turned into a numeric representation. One kind of representation we are already familiar with is one-hot-encoding. We can apply one-hot-encoding to this data too, and it would look like this:

- get a list of all the unique words across **all** the reviews (we call this a **corpus**) and create dummy columns for each word
- for each review let the word dummy column be 1 if the word is present in the review and 0 otherwise

The problem with the above approach is that we end up with lots of dummy columns (one per word) and for any given review the majority of the encodings would be zero. One-hot encoding can still be a good idea, but we need to find a way to representing in a **sparse** format instead.

One approach is to sort the corpus by most frequent to least frequent. We then index the words using this order, starting with index 1, so that the index 1 would encodes most frequent word in the corpus, 2 would be the second most frequent word, etc. We can download the **word to index dictionary** used by the IMBD data set to see this.

In [ ]:
word_to_index = tf.keras.datasets.imdb.get_word_index()
print("There are {:,} in the corpus and indexing starts at {}.".format(
    len(word_to_index.keys()), min(word_to_index.values())))

Let's print the word and index for the first 10 words. Note that `word_index` is a dictionary, so it is not necessarily sorted in the order of frequency.

In [ ]:
first_10_keys = list(word_to_index.keys())[:10]

for k in first_10_keys:
    print(f"For word \"{k}\" \t the index is {word_to_index[k]}.")

We now take this word to index and add a few more "words" to it. These are not really words, but markers that are used by the algorithm:

By convention index 0 though 3 are reserved for the some special markers shown below. What these markers are will become clear later. We make room for these markers in the word to index dictionary by pushing the current indexes out.

In [ ]:
word_to_index = {k:(v + 3) for k, v in word_to_index.items()}
word_to_index["<PAD>"] = 0
word_to_index["<START>"] = 1
word_to_index["<UNK>"] = 2
word_to_index["<UNUSED>"] = 3

Let's now create a **reverse dictionary**, that is one where we switch keys and values.

In [ ]:
index_to_word = {value: key for key, value in word_to_index.items()}

In [ ]:
for i in range(10):
    print(f"For index {i} the word is \"{index_to_word[i]}\".")

### Exercise

Let's look at the first review in the data:

In [ ]:
print(x_train[0])

- Write a function named `decode_review` that takes as input a review like the above one and uses `index_to_word` to return the actual review as a string. 

- We now decode the first review in the data using your function:

- What do you think the presence of `<UNK>` in the review means?

### End of exercise

We saw how we need to force all images in a convolutional network to have a same size by choosing a size and then padding smaller images around the edges until they have the same size. We need to do the same with our reviews: we need to pick a maximum number of words a review can have, and (1) trim longer reviews and (2) pad shorter reviews. Foretunately, `keras` already has a function for doing that.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_review_length = 300
x_train_padded = pad_sequences(x_train, maxlen = max_review_length)
x_test_padded = pad_sequences(x_test, maxlen = max_review_length)

Let's see what the first review looks like once it's padded.

In [ ]:
x_train_padded[0]

We can now train our model. Instead of the vanilla recurrent neural network, we use a fancier kind called an LSTM. An LSTM is a recurrent network that is better at retaining long-term information, which is very important when dealing with language data. A vanilla RNN on the other hand tends to put much more emphasis on recent words.

In [ ]:
embedding_vecor_length = 250

model = keras.models.Sequential()
model.add(keras.layers.Embedding(num_of_words, embedding_vecor_length, input_length = max_review_length))
# model.add(keras.layers.LSTM(32, return_sequences = True))
model.add(keras.layers.LSTM(32))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Let's now train our model. This is a model lots of parameters and can take a while to train.

In [ ]:
model.fit(x_train_padded, y_train, validation_data = (x_test_padded, y_test), epochs = 2, batch_size = 128)

Now that we have trained our model, lets look at the testing accuracy.

In [ ]:
# Evaluate model
scores = model.evaluate(x_test_padded, y_test)
print("Accuracy: {:.2f}%".format(scores[1]*100))

### Exercise

- What does the model predict for the first review in the test data? HINT: you may need to use `reshape`.

- Read the first review in the test data using the function you wrote earlier to decode reviews. Does the prediction seem good?

- Print 5 examples in the test data where the prediction is correctly classified, and 5 examples where the prediction does not match the label, i.e. where the model mis-classified. Read the reviews and see if the mis-classified reviews seem less clear-cut than the correctly classified reviews. To help you with this, we show you in the following cell how you can test whether the first review in the test data was correctly classified or not.

In [ ]:
pred = model.predict(x_test_padded[0].reshape(1, -1))[0][0]
obsv = y_test[0]
if ((pred > 0.5) & (obsv > 0.5)) | ((pred < 0.5) & (obsv < 0.5)):
    print("Correctly classfied.")
else:
    print("Incorrectly classified.")

### End of exercise

One of the impressive things about neural networks is how changes in their architectures can suddenly open them up to solving a new set of problems. Convolutional neural networks and recurrent neural networks are just two examples. In many ways the journey is only beginning.